In [6]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
import PIL
import numpy as np
import pandas as pd
import sys
from pathlib import Path

In [7]:
read_range = (3840,3999)
im_path = "data/real_progress_marked/"

In [8]:
def show(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    display(PIL.Image.fromarray(img))
def show_plt(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

In [9]:
images_list = os.listdir(im_path)

In [10]:
def thresh(gray, thrsh):
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    ret, th = cv2.threshold(blur, thrsh, 255, cv2.THRESH_BINARY)
    return th
def saveImg(directory,i, img):
    Path(directory).mkdir(parents=True, exist_ok=True)
    path = directory+str(i)+".jpg"
    cv2.imwrite(path,img)
    return True
def getPerspectiveTransform(points, col_dis):
    output_size = 4000
    startx = output_size / 2 - col_dis[0]/2
    endx = output_size / 2 + col_dis[0]/2

    starty = output_size / 2 - col_dis[1]/2
    endy = output_size / 2 + col_dis[1]/2

    targetPoints = [(startx, starty), (endx, starty), (startx, endy), (endx, endy)]

    empty_img = np.ones((output_size, output_size), dtype=np.uint8)
    
    pst1 = np.float32(points)
    pst2 = np.float32(targetPoints)
    '''
    A perspective transform relates two points in the following manner:  
    [x']   [m00 m01 m02] [x]
    [y'] = [m10 m11 m12] [y]
    [1]    [m20 m21 m22] [1]
    Where (x,y) are the original 2D point coordinates, and (x', y') are the transformed coordinates.
    '''
    matrix = cv2.getPerspectiveTransform(pst1, pst2)
    return matrix
def find_contours(bimg, min_size):
    c, h = cv2.findContours(bimg, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    c = [a for a in c if a.shape[0] > min_size]
    return c
def contours_area_count(c):
    sum = 0
    for i in c:
        area = cv2.contourArea(i)
        sum += area
    return sum

In [11]:
def main():
    frame_numbers = []
    areas = []
    pathArr = []
    method = 'REAL'
    prv_area = 0
    for img_file_name in images_list:
        print(img_file_name)
        img = cv2.imread(im_path+img_file_name)
        red_channel = img[:,:,2]
        th = thresh(red_channel, 250)
        points = [(791, 534), (1071, 530), (583, 672), (1289, 658)]
        matrix = getPerspectiveTransform(points, (850,900))
        output_size =4000
        th_transformed = cv2.warpPerspective(th, matrix, (output_size, output_size))
        c = find_contours(th_transformed, 300)
        area_count = contours_area_count(c)
        if prv_area > area_count:
            area_count = prv_area
        prv_area = area_count
        areas.append(area_count)
        i = img_file_name.split('j')[0]
        frame_numbers.append(i)

        th_rgb = cv2.cvtColor(th_transformed, cv2.COLOR_BAYER_BG2BGR)
        cv2.drawContours(th_rgb, c, -1, (0, 255, 0), 2)
        threshImgsDir = 'flow_results/'+method+"/thresh/"
        saveImg(threshImgsDir, i, th_rgb)
        pathArr.append(str(i)+'.jpg')
    
    df = pd.DataFrame({'Frame':frame_numbers, 'Area':areas, 'path': pathArr})
    csvPath = 'export_dataframe_'+method+'.csv'
    df.to_csv(csvPath, index=False)
main()

3840j.jpg


ValueError: too many values to unpack (expected 2)